In [1]:
import win32com.client
from datetime import date
import numpy as np
import time
from os import listdir
from os.path import isfile, join
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit
import visa # provides generic usb interface to the powermeter4
from ThorlabsPM100 import ThorlabsPM100  # actually knows how to talk to the powermeter
%matplotlib notebook

## set basic parameters below

In [2]:
### set folder here

# construct Packer-xkcd-compliant datestring
datestring = date.today().isoformat()

# type your name here (formated as in F drive)
#name = 'rlees'

# folder = r"F:\Data\{0}\{1}\PowerMeasurements".format(name, datestring)
folder = r"C:\PowerMeasurements\{0}".format(datestring)

if not os.path.exists(folder):
    os.makedirs(folder)
    
folder

'C:\\PowerMeasurements\\2022-04-06'

In [3]:
class DigitalPowerMeasurer:
    """
    This class takes analog devices on an AI card, a usb connection to the power meters, 
    a PrairieLink, and xmls and gpls to definethe SLM experiment.
    It then fires the experiment once to find the best power range, 
    initializes the analog devices, starts the SLM experiments and listens to the analog inputs.
    returns a power range array and the analog input array."""
    
    def __init__(self,prairie_link, visa_resource_manager, power_meter_list):
        
        # store configruation
        self.pl = prairie_link
        self.rm = visa_resource_manager
        self.num_power_meters = len(power_meter_list)
        
        # create power_meters
        self.instruments = []
        self.power_meters = []
        for i in range(self.num_power_meters):
            self.instruments.append(rm.open_resource(power_meter_list[i]))
            self.power_meters.append(ThorlabsPM100(inst=self.instruments[i]))
        
    def measure(self,laser_name,wavelength,laser_power,window_length=5):
        """
        laser name needs to match a laser name in PrairieView
        Wavelength needs to be accessible trhough the PrairieView 2P laser tab
        take {laser_name:laser_power[PV]} dictionary and perform measurement with this power for 3s"""
        
        voltage_array = np.zeros(self.num_power_meters)
        
#         pl.SendScriptCommands('-SetLaserPower ' + laser_name + ' ' + str(laser_power))
        pl.SendScriptCommands('-sts laserPower ' + str(laser_power) + ' "' + laser_name + '"')
        time.sleep(window_length)
        
        for i in range(self.num_power_meters):
            # set the power meter wavelength
            self.power_meters[i].sense.correction.wavelength = wavelength
            
            # use USB correction to measure current power
            voltage_array[i] = self.power_meters[i].read
        
        pl.SendScriptCommands('-SetLaserPower ' + laser_name + ' ' + str(0))
        
        return voltage_array

In [4]:
def pockel_curve(x,I,k,phi):
    return I * np.cos(k*x + phi)**2

def arctangent(x, phi, I_0, I, k):
    return I * np.arctan(k*(x + phi)) + I_0

def sigmoid(x, a, b):
    y = 1 / (1 + np.exp(-b*(x-a)))
    return y

In [5]:
# Start PrairieLink
pl = win32com.client.Dispatch('PrairieLink.Application')
print('object created')

# Connect to Client
pl.Connect()
print(pl.Connected)

object created
<bound method Connected of <COMObject PrairieLink.Application>>


In [6]:
# Search for PowerMeters
rm=visa.ResourceManager()
resource_list = rm.list_resources()
print(resource_list)

# create powermeter, assume that it is the first object showing up in the above list
pm100 = DigitalPowerMeasurer(pl,rm,[resource_list[0]])

('USB0::0x1313::0x8079::P1003280::INSTR', 'ASRL7::INSTR', 'ASRL12::INSTR', 'ASRL13::INSTR', 'ASRL14::INSTR', 'ASRL15::INSTR')


# MAKE sure you:
## Open maintenance window in PV: Ctrl + M or Tools > Maintenance
## check PMT shutters are closed
## open laser shutters
## center galvos
## check you are in resonant galvo mode (prarie view)
## plug in power meter if low battery
## check the conoptics bias voltage is set to 005
## SET ZOOM TO 8X TO REDUCE VARIABILITY DUE TO GALVO SCANNING

In [21]:
laser_name = 'Imaging'
wavelength = 920

#pl.SendScriptCommands('-SetMultiPhotonWavelength' + ' ' + str(wavelength) + ' ' + str(1))
#Allow Chameleon to change its colours
#time.sleep(15)

powerspace = np.arange(0,1100,100)

len_powerspace = len(powerspace)

voltage_array = np.zeros(len_powerspace)

for i in range(len_powerspace):
    print(powerspace[i])
    voltage_array[i] = pm100.measure('Imaging', wavelength, powerspace[i],6)
#     time.sleep(3)
    print(voltage_array[i])

0


C:\Users\User\Anaconda3\lib\site-packages\pyvisa\resources\messagebased.py:568: FutureWarning: ask is deprecated and will be removed in 1.10, use query instead.
  '1.10, use query instead.', FutureWarning)


-0.000474126748
100
0.000326129026
200
-0.000674250943
300
0.00012600483
400
0.000726256927
500
0.000526253192
600


KeyboardInterrupt: 

In [ ]:
guess = (0.14,1/500,1)
popt, pcov = curve_fit(pockel_curve,powerspace,voltage_array,guess)

fig = plt.figure()
ax = plt.subplot(111)

ax.plot(powerspace,1000*voltage_array,linestyle='None',marker='o',label=laser_name + ' Power-on-Table')
ax.plot(powerspace,1000*pockel_curve(powerspace,*popt),label=r"cos$^{2}$ Fit" + "\n"
         r"Peak Power: {:.2f} [mW]".format(popt[0]*1000) + "\n" +
         "Offset: {:.2f} [PV]".format(popt[2]))

#numerically get the PV power for 50mW
test_PVs = np.linspace(0,500,1000)
power = []
for i in test_PVs:
    
    power.append(pockel_curve(i,*popt) * 1000)
    
fifty_diff = [50 - i for i in power]
fifty_IND = np.argmin(np.abs(fifty_diff))
fifty_PV = np.round(test_PVs[fifty_IND],2)


# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
legend = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel("Voltage [PV]")
plt.ylabel("Power [mW]")
plt.title("Packer1 Power Measurement \n" + str(wavelength) + " nm, " + datestring + ' 50mW is {}'.format(fifty_PV))

# construct Packer-xkcd-compliant datestring
datestring = date.today().isoformat()
operator = "mloidolt"

os.chdir(folder)
plt.savefig(datestring+'_'+laser_name+'_'+str(wavelength)+".pdf",bbox_extra_artists=(legend,), bbox_inches='tight')
np.save(datestring+'_'+laser_name+'_'+str(wavelength)+'_'+'powerspace',powerspace)
np.save(datestring+'_'+laser_name+'_'+str(wavelength)+'_'+'voltage_array',voltage_array)
np.save(datestring+'_'+laser_name+'_'+str(wavelength)+'_'+'popt',popt)
np.save(datestring+'_'+laser_name+'_'+str(wavelength)+'_'+'pcov',pcov)

In [ ]:
# #TEST CELL SARAH 2019-07-24
# You can use this function to test the consistency of power measurements over repeats
# # powerspace_small = np.arange(0,300,20)
# # powerspace_large = np.arange(300,1100,100)
# # powerspace = np.append(powerspace_small,powerspace_large)
# powerspace = np.arange(40,300,20)
# repeats = 30
# wavelength = 765
# timesleep=5

# def powercurve_repeats(repeats,powerspace,wavelength,timesleep): 
#     # repeats = how many iterations of the power curve to do
#     # powerspace = 1d array containing the PV values to test
#     #timesleep = time (s) in between iterations
#     # wavelength = laser wavelength to test
#     assert wavelength == 765 or wavelength == 920
#     assert np.max(powerspace) < 1000
#     #Repeat the power curves n number of times
#     voltage_arrays = []
#     fifty_PVs = []
#     for n in range(repeats):
#         voltage_array = np.zeros(len(powerspace))
#         for i in range(len(powerspace)):
#             voltage_array[i] = pm100.measure('Imaging', wavelength, powerspace[i],6)
#             time.sleep(timesleep)
#         voltage_arrays.append(voltage_array)
#         #Curve fitting
#         guess = (0.14,1/500,1)
#         popt, pcov = curve_fit(pockel_curve,powerspace,voltage_array,guess)

#         #numerically get the PV power for 50mW
#         test_PVs = np.linspace(0,500,1000)
#         power = []
#         for i in test_PVs:
#             power.append(pockel_curve(i,*popt) * 1000)
#         fifty_diff = [50 - i for i in power]
#         fifty_IND = np.argmin(np.abs(fifty_diff))
#         fifty_PV = np.round(test_PVs[fifty_IND],2)
#         fifty_PVs.append(fifty_PV)
#         print("repeats completed:",n+1)
#         time.sleep(timesleep)
#     return(fifty_PVs,voltage_arrays)

In [ ]:
#fiftyPVs_765, voltage_arrays_765 = powercurve_repeats(repeats,powerspace,wavelength,timesleep)

# Open blink
# open uncaging laser shutter in PV
# be very careful not to go above 800 PV if using uncaging path 
# Pmax = 5W for the thermosensor
# be even MORE careful if using the wand Pmax = 0.5W
# Make sure at 1x zoom
# overdrive enabled
# (under Test masks) change phase mask to 2018-11-07_SLMSingleSpot_PhaseMask_PowerMeasurement


In [ ]:
## set the uncaging path to measure here

#uc_path = 'uncaging_no_slm'
uc_path = 'uncaging_slm_single_spot' 

In [ ]:
uc_path

In [ ]:
# dont try hack this to make it faster, it causes errors

laser_name = 'Uncaging'
wavelength = 1030

if uc_path == 'uncaging_no_slm':
    powerspace = np.arange(200,750,50)
elif uc_path == 'uncaging_slm_single_spot':
    powerspace = np.arange(0,800,80)
else:
    raise('invalid uncaging path')


len_powerspace = len(powerspace)

voltage_array = np.zeros(len_powerspace)

for i in range(len_powerspace):
    print(powerspace[i])
    voltage_array[i] = pm100.measure(laser_name, wavelength, powerspace[i],window_length=7) 
    print(voltage_array[i])
#     time.sleep(3)
    
assert max(voltage_array) < 10, 'if power meter is not allowed to reset long enough between measurements or has low battery, an error can be raised here'

In [ ]:
guess = (0.14,1/1000,1)
popt, pcov = curve_fit(pockel_curve,powerspace,voltage_array,guess)

fig = plt.figure()
ax = plt.subplot(111)

ax.plot(powerspace,1000*voltage_array,linestyle='None',marker='o',label=laser_name + ' Power-on-Table')
ax.plot(powerspace,1000*pockel_curve(powerspace,*popt),label=r"cos$^{2}$ Fit" + "\n"
         r"Peak Power: {:.2f} [mW]".format(popt[0]*1000) + "\n" +
         "Offset: {:.2f} [PV]".format(popt[2]))

#numerically get the PV power for 50mW
test_PVs = np.linspace(0,500,1000)
power = []
for i in test_PVs:
    
    power.append(pockel_curve(i,*popt) * 1000)
    
fifty_diff = [50 - i for i in power]
fifty_IND = np.argmin(np.abs(fifty_diff))
fifty_PV = np.round(test_PVs[fifty_IND],2)


# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
legend = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel("Voltage [PV]")
plt.ylabel("Power [mW]")
plt.title("Packer1 Power Measurement \n" + str(wavelength) + " nm, " + datestring + ' 50mW is {}'.format(fifty_PV))

# construct Packer-xkcd-compliant datestring
datestring = date.today().isoformat()
operator = "mloidolt"

os.chdir(folder)
plt.savefig(datestring+'_'+laser_name+'_'+str(wavelength)+".pdf",bbox_extra_artists=(legend,), bbox_inches='tight')
np.save(datestring+'_'+laser_name+'_'+str(wavelength)+'_'+'powerspace',powerspace)
np.save(datestring+'_'+laser_name+'_'+str(wavelength)+'_'+'voltage_array',voltage_array)
np.save(datestring+'_'+laser_name+'_'+str(wavelength)+'_'+'popt',popt)
np.save(datestring+'_'+laser_name+'_'+str(wavelength)+'_'+'pcov',pcov)

In [ ]:
def arctangent(x, phi, I_0, I, k):
    return I * np.arctan(k*(x + phi)) + I_0

def sigmoid(x, a, b):
    y = 1 / (1 + np.exp(-b*(x-a)))
    return y

def fsigmoid(x, a, b):
    return 1.0 / (1.0 + np.exp(-a*(x-b)))

In [ ]:
datestring = date.today().isoformat()
guess = (-700,500,300,0.02)
guess = (0,0,300,0.02)

popt, pcov = curve_fit(arctangent,powerspace,voltage_array,guess,method='dogbox')
#guess_sig = (650,0.3)
#popt, pcov = curve_fit(sigmoid,powerspace,voltage_array,guess_sig,method='dogbox')
#popt, pcov = curve_fit(fsigmoid, powerspace, voltage_array, method='dogbox', bounds=([0.1, 200.],[0.01, 900.]))
fig = plt.figure()
ax = plt.subplot(111)

ax.plot(powerspace,voltage_array,linestyle='None',marker='o',label=uc_path)
ax.plot(powerspace,arctangent(powerspace,*popt))#,label=r"arctan Fit" + "\n"
         #r"Peak Power: {:.2f} [mW]".format((popt[2]+popt[1])*1000) + "\n" +
         #"Offset: {:.2f} [PV]".format(popt[0]))

#numerically get the PV power for 50mW
test_PVs = np.linspace(200,500,1000)
power = []
for i in test_PVs:    
    power.append(arctangent(i,*popt) * 1000)
    
fifty_diff = [50 - i for i in power]
fifty_IND = np.argmin(np.abs(fifty_diff))
fifty_PV = np.round(test_PVs[fifty_IND],2)

# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
legend = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xlabel("Voltage [PV]")
plt.ylabel("Power [mW]")
#plt.title("Packer1 Power On Sample \n" + str(wavelength) + " nm " + ' ' + uc_path + "\n" + datestring)
plt.title("Packer1 Power Measurement \n" + str(wavelength) + " nm, " + datestring + ' 50mW is {}'.format(fifty_PV))

# construct Packer-xkcd-compliant datestring
datestring = date.today().isoformat()
operator = "mloidolt"

os.chdir(folder)
plt.savefig(datestring+'_'+uc_path+'_'+str(wavelength)+".pdf",bbox_extra_artists=(legend,), bbox_inches='tight')
np.save(datestring+'_'+uc_path+'_'+str(wavelength)+'_'+'powerspace',powerspace)
np.save(datestring+'_'+uc_path+'_'+str(wavelength)+'_'+'voltage_array',voltage_array)
np.save(datestring+'_'+uc_path+'_'+str(wavelength)+'_'+'popt',popt)
np.save(datestring+'_'+uc_path+'_'+str(wavelength)+'_'+'pcov',pcov)
sns.despine()

print(powerspace);
print(voltage_array);

# In MATLAB, plot all laser power measurements over time
#powerPlotTime('E:\PowerMeasurements', 0.5)
#Thanks Rob

In [ ]:
# folder = r"F:\Data\mloidolt\2018-09-05\2018-09-05_PowerMeasurements"

# os.chdir(folder)

# datestring = date.today().isoformat()

# laser_name = 'Uncaging'
# wavelength=1030

# powerspace = np.load("2018-09-05_Uncaging_1030_powerspace.npy")
# voltage_array = np.load("2018-09-05_Uncaging_1030_voltage_array.npy")
# stable_popt = np.load("2018-09-05_Uncaging_1030_popt.npy")

# Y = I/2 * 680
# X = PV

# old_popt = np.load(r"F:\Data\mloidolt\2018-08-23\2018-08-23_PowerMeasurements\2018-08-23_Uncaging_1030_popt.npy")
# powerspace = np.arange(300,1000,30)

# guess = (-700,500,300,0.02)

# popt, pcov = curve_fit(arctangent,X,Y,guess)
# print(popt)
# print(old_popt)

# fig = plt.figure()
# ax = plt.subplot(111)

# #plt.plot(powerspace,arctangent(powerspace,*old_popt)/multiplicator)
# ax.plot(X,Y,linestyle='None',marker='o',color='blue',label=laser_name + ' Peak Power')
# ax.plot(powerspace,arctangent(powerspace,*popt),color='blue',label=r"arctan Fit" + "\n"
#          r"Peak Power: {:.2f} [mW]".format((popt[2]+popt[1])*1000) + "\n" +
#          "Offset: {:.2f} [PV]".format(popt[0]))
# ax.plot(powerspace,1000*voltage_array,linestyle='None',marker='o',label=laser_name + ' Power afer 5s',color='green')
# ax.plot(powerspace,1000*arctangent(powerspace,*stable_popt),label=r"arctan Fit" + "\n"
#          r"Peak Power: {:.2f} [mW]".format((popt[2]+popt[1])*1000) + "\n" +
#          "Offset: {:.2f} [PV]".format(old_popt[0]),color='green')

# # Shrink current axis by 20%
# box = ax.get_position()
# ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# # Put a legend to the right of the current axis
# legend = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
# plt.xlabel("Voltage [PV]")
# plt.ylabel("Power [mW]")
# plt.title("Packer1 Power Measurement \n" + str(wavelength) + " nm, " + datestring)